### Что такое сокет? 

Сокет - низкоуровневая абстракция для отправки  и получения данных. Сокет поддерживает две операции - отправку и получение байтов. 

По умолчанию сокеты блокирующие, т. е. на все время ожидания ответа от сервера приложение останавливается или *блокируется* и не может ничего делать, пока не придут данные от сервера. Сокеты могут работать в *неблокирующем* режиме. Когда байты будут получены, ОС уведомит об этом, а пока они не получены, приложение может выполнять другие задачи. Для реализации такой схемы применяются различные системы в уведомлений в разных ОС:
- kqueue - FreeBSD и MacOS;
- epoll - Linux;
- IOCP - Windows

Эти системы лежат в основе модели конкуретности в asyncio. Встретив операцию ввода-вывода, интерпретатор передает ее на попечение системы уведомления, входящей в состав ОС. По завершении  этой операции система пробуждает задачу, ожидающую результата, после чего выполняется код, следующий за этой операцией.


### Select

Каждый раз, когда принимается соединение с помощью системного вызова accept, мы получаем новый файловый дескриптор, представляющий это соединение. Если есть некий веб-сервер, то у него может быть открыто одновременно тысячи соединений и нужно их обрабатывать на предмет новых запросов. Вместо того чтобы перебирать их в цикле (что долго), можно использовать системные вызовы, которые позволяют отслеживать множество файловых дескрипторов: ```poll```, ```epoll``` и ```select```.  - системный вызов, который возвращает список готовых к обработке дескрипторов ввода-вывода (файлов, сокетов и др.). 

### Файловый дескриптор

Это неотрицательное целое число. Когда создается новый поток ввода-вывода, ядро возвращает процессу, создавшему поток ввода-вывода, его файловый дескриптор.

По умолчанию 0 - stdin, 1 - stdout, 2 - stderr. Обычно номера дескрипторов выделяются последовательно, так что если в программе открыть какой-либо файл на чтение или запись, то скорее всего это будет номер 3. Файловые дескрипторы могут принимать значения от 0 до OPEN_MAX. Согласно POSIX.1, значение OPEN_MAX равно 19. В реальных ОС это значение может быть больше.

### Системные вызовы

**listen** - формирует очередь входящих подключений. Параметр ```backlog``` указывает на максимальный размер этой очереди - количество соединений, которое сервер может одновременно удерживать в этой очереди.

**accept** - извлекает первый запрос на соединение из очереди входящих подключений и создает новый подключенный сокет. 

# Asyncio

При выполнении асинхронного вызова функции мы делаем запрос на выполнение вызова функции, но не ждем завершения этого вызова. Вызывающая стороны после выполнения вызова может продолжать работу и решать другие задачи. 

Когда использовать асинхронщину:
- микросервисы (I/O bound, не CPU-bound)
- долгоживущие соединения (websocket, раздача файлов)
- есть производительная инфраструктура (шардированные базы, кеши, write-heavy очереди)
- экономия ресурсов серверов

Когда НЕ использовать асинхронщину:
- CPU-bound
- боттлнек (узкое место) в инфраструктуре (напр один инстанс базы данных)

### Когда использовать многопоточность, когда многопроцессность и когда асинхронность? 

В нескольких потоках можно обрабатывать одновременно несколько задач ввода-вывода. Например, когда нужно обрабатывать соединения с несколькими клиентами. Когда сервер получает соединение, запускает функцию обработчика в отдельном потоке. Функция обработчика взаимодействует с клиентом путем чтения и записи в сокет клиента. При чтении из сокета поток зависает, пока клиент что-то не отправит. В этот момент GIL отпускается и его может захватить другой поток, этот поток может обрабаывать соединение с другим клиентом. 

### Глобальная блокировка интерпретатора

GIL не дает Python-процессу исполнять более одной команды байт-кода в каждый момент времени. Даже если на многоядерной машине имеется несколько потоков, интерпретатор сможет в каждый момент исполнять только один поток, содержащий написанный на питоне код. 

Примечание: Многопроцессные приложения могут конкурентно выполнять несколько команд байт-кода, так как у каждого питоновского процесса свой GIL.

Вызов системных функций ввода-вывода является наиболее распространенным вариантом использования для выпуска GIL, но он также может быть полезен перед вызовом длительных вычислений, которым не требуется доступ к объектам Python, таким как сжатие или криптографические функции, работающие с буферами памяти. Например, стандарт zlib и hashlib модули освобождают GIL при сжатии или хешировании данных.

В Asyncio используется тот факт, что операции ввода-вывода освобождают гил, что позволяет реализовать конкурентность даже в одном потоке. При работе с asyncio мы создаем объекты корутин, которые можно представить себе как облегченный поток. Ожидая завершения сопрограмм, занимающихся вводом-выводом, мы можем выполять другой питоновский код. 

На разных системах существуют и другие альтернативы select — poll, epoll, devpoll, kqueue — использование которых для тех или иных случаев может быть выгоднее, чем select.Чтобы унифицировать процесс получения готовых к работе дескрипторов ввода-вывода в Python был создан модуль ```selectors```. Этот модуль обеспечивает высокоуровневое и эффективное мультиплексирование ввода-вывода, построенное на ```select``` примитивах модуля. Вместо этого пользователям рекомендуется использовать этот модуль, если они не хотят точного контроля над используемыми примитивами уровня ОС.

Он определяет ```BaseSelector``` абстрактный базовый класс вместе с несколькими конкретными реализациями (```KqueueSelector```, ```EpollSelector```), которые можно использовать для ожидания уведомления о готовности ввода-вывода для нескольких файловых объектов. «Файловый объект» относится к любому объекту с ```fileno()``` методом или необработанному файловому дескриптору.

### Awaitable объекты 

Это объекты, ожидающие результатов. Есть три основных типа объектов, которые можно запускать оператором await в асинхронном коде:
- корутина (сопрограмма) - функция, определенная с оператором async
- таска (задача) - сопрограмма, которая планируется для выполнения в будущем, как только это станет возможным
- футура - объект уже запущенной сопрограммы с еще не полученными или промежуточными результатами

Функцию корутины можно рассматривать как фабрику для создания объектов корутины. Вызов функции корутины не приводит к выполнению кода, написанного пользователем. Вместо этого в ходе такого вызова лишь создаётся и возвращается объект корутины.